In [91]:
# script to read in awas data for flights 5 - 14
# remove samples in SH, near guam, and where tracer values < 0 

In [1]:
import xarray as xr
import numpy as np 
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
### ---------------------- READ IN DATA  ---------------------- ###
# path to data
mypath = '/Volumes/scdrive2/contrast_merge_data/contrast_awas_merge/*.nc'

# read in merge all flights 
contrast_init = xr.open_mfdataset(mypath, combine='by_coords')
# drop the variables without category metadata (no instrument)
contrast_init = contrast_init.drop('DOAS_BrO').drop('DOAS_delta_BrO')

# change coordinate from "Time" to "Time_UTC"
contrast_init.coords['Time_UTC'] = (contrast_init.Time_UTC)

In [3]:
contrast_init

<xarray.Dataset>
Dimensions:                           (Time: 657)
Coordinates:
  * Time                              (Time) datetime64[ns] 2014-01-22T00:20:25 ... 2014-02-25T07:17:05
    Time_UTC                          (Time) datetime64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
Data variables:
    FLT_HH                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    FLT_MM                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    FLT_SS                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    UTC_HH                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    UTC_MM                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    UTC_SS                            (Time) timedelta64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
    GALT_A                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGALT                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGEOIDHT                          (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGLAT                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGLON                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGQUAL                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGSPD                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    PITCH                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    RICE                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    ROLL                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    THDG                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    VISBC                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    VISTC                             (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    ATX                               (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    CONCD_RWOO                        (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    CONCU100_RWOI                     (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    CONCU500_RWOI                     (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    CONCU_RWOI                        (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    CONCV_VXL                         (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DBARD_RWOO                        (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DBARU_RWOI                        (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DPXC                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DP_DPL                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DP_DPR                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    DP_VXL                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    EW_DPL                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    EW_DPR                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    EW_VXL                            (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    LATC                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    LONC                              (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    MR                                (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    PALT             

In [4]:
### --- what instruments do each data variable come from?
#for ii in contrast_init.data_vars:
#    print(contrast_init[ii].attrs['Category'])

### --- print each awas measured tracer 
for ii in contrast_init.data_vars:
    if contrast_init[ii].attrs['Category'] == 'AWAS':
        print(contrast_init[ii].long_name)

C2H6_Ethane
C2H2_Ethyne
C3H8_Propane
C4H10_Isobutane
C4H10_nButane
C5H12_Isopentane
C5H12_nPentane
C5H8_Isoprene
C6H6_Benzene
OCS_Carbonyl_sulfide
C2H6S_Dimethyl_sulfide
CCl2F2_CFC_12
CCl3F_CFC_11
CCl2FCCl2F_CFC112
CCl3CClF2_CFC_112a
C2Cl3F3_CFC_113
C2Cl2F4_CFC_114
CHClF2_HCFC_22
CH3CClF2_HCFC_142b
CH3CCl2F_HCFC_141b
CH2FCF3_HFC_134a
C4H5F5_HFC365mfc
CH3Cl_Methyl_chloride
CH3Br_Methyl_bromide
CH3I_Methyl_iodide
CH2Cl2_Methylene_chloride
CHCl3_Chloroform
CH3CCl3_Methyl_chloroform
CCl4_Carbon_tetrachloride
C2H4Cl2_1_2_Dichloroethane
C2HCl3_Trichloroethylene
C2Cl4_Tetrachloroethylene
C6H5Cl_Chlorobenzene
CBrClF2_Halon_1211
C2Br2F4_Halon_2402
CH2BrCl_Bromochloromethane
CH2Br2_Methylene_bromide
CHBrCl2_Bromodichloromethane
CHBr2Cl_Dibromochloromethane
CHBr3_Bromoform
CH3ONO2_Methyl_nitrate
C2H5ONO2_Ethyl_nitrate
C3H7NO3_Isopropyl_nitrate
C4H9NO3_2_Butyl_nitrate
C4H9NO3_n_Butyl_nitrate
C5H11NO3_3_pentyl_nitrate
C5H11NO3_2_pentyl_nitrate


In [5]:
### --- subset the data so that we only keep awas tracers that we want 
# empyt dataset to fill with awas data 
awas_init = xr.Dataset()
# fill with anything that has 'awas' as the category
for ii in contrast_init.data_vars:
    if contrast_init[ii].attrs['Category'] == 'AWAS':
        awas_init[ii] = contrast_init[ii]
        
### --- remove any AWAS tracers we dont use in TTS paper (40 awas total used)
awas_trc_prelim = awas_init.drop('C5H12_Isopentane').drop('C5H8_Isoprene'). \
    drop('CH3I_Methyl_iodide'). \
    drop('C2HCl3_Trichloroethylene').drop('CH3ONO2_Methyl_nitrate'). \
    drop('C5H11NO3_3_pentyl_nitrate').drop('C5H11NO3_2_pentyl_nitrate')
print(len(awas_trc_prelim))

### --- awas aircraft parameters and location 
awas_param_init = xr.merge([contrast_init.GGALT, contrast_init.GGLAT, contrast_init.GGLON])
                            #contrast_init.Time_UTC])
awas_param_init               
    #contrast_init.WIC, contrast_init.PALT, contrast_init.RHUM])

40


<xarray.Dataset>
Dimensions:   (Time: 657)
Coordinates:
  * Time      (Time) datetime64[ns] 2014-01-22T00:20:25 ... 2014-02-25T07:17:05
    Time_UTC  (Time) datetime64[ns] dask.array<chunksize=(59,), meta=np.ndarray>
Data variables:
    GGALT     (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGLAT     (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>
    GGLON     (Time) float32 dask.array<chunksize=(59,), meta=np.ndarray>

In [6]:
### ---------------------- CONFINE TO NH  ---------------------- ###

# take a look at where values for latitude lie - NH, SH, or nan
#southhem = np.where(awas_param_init.GGLAT.values < 0)
studyarea = np.where((awas_param_init.GGLAT.values > 0) & (awas_param_init.GGLAT.values < 25))
print(np.size(studyarea))
#nullinit = np.isnan(awas_param_init.GGLAT.values).sum()

# keep only NH values, rewrite to awas_param 
awas_param_noblcorr = xr.Dataset()
for ii in awas_param_init.data_vars:
    param = awas_param_init[ii]
    nh_only = param[studyarea].values 
    awas_param_noblcorr[ii] = nh_only
    
# select NH only for all tracers 
awas_trc_twp = xr.Dataset()
for ii in awas_trc_prelim.data_vars:
    trc = awas_trc_prelim[ii]
    nh_only = trc[studyarea]
    awas_trc_twp[ii] = nh_only

543


/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """


In [7]:
### ---------------------- REMOVE NULL VALUES, < 0 VALUES ---------------------- ###
# ignore comparison to nan warnings (for where trc<0)
np.warnings.filterwarnings('ignore')

### null values (-777, -888), make = nan 
awas_trc_noblcorr = xr.Dataset()
for ii in awas_trc_twp.data_vars:
    trc = awas_trc_twp[ii]
    trc_vals = trc.values
    
    # -------- locations for fills
    llod = awas_trc_twp[ii].attrs['LLOD']
    ulod = awas_trc_twp[ii].attrs['ULOD']
    
    # -------- make those locations nan and save to variable 
    #trc_masked = trc.where((trc.values != llod) & (trc.values != ulod))
    trc_masked = np.ma.masked_where(trc_vals < 0, trc_vals)
    trc_masked = np.ma.masked_where(trc_vals == llod, trc_masked)
    trc_masked = np.ma.masked_where(trc_vals == ulod, trc_masked)
    trc.values = trc_masked 
    
    awas_trc_noblcorr[ii] = trc

In [8]:
### ---------------------- ADD LIFETIMES AS AN ATTRIBUTE ---------------------- ###
### --- add attribute to each (BL, tropo, and UT lifetimes)
# read in from xls file to make dataframe
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)

# fill in lifetime attributes using that dataframe 
for ii in enumerate(awas_trc_noblcorr.data_vars):
    myind = ii[0]
    trc = ii[1]
    awas_trc_noblcorr[trc].attrs['BL_lifetime'] = awas_lifetimes['BL_tau'][myind]
    awas_trc_noblcorr[trc].attrs['UT_lifetime'] = awas_lifetimes['UT_tau'][myind]
    awas_trc_noblcorr[trc].attrs['TROPO_lifetime'] = awas_lifetimes['TROPO_tau'][myind]

In [9]:
### ---------------------- REMOVE BL SAMPLES NEAR GUAM ---------------------- ###
# ~111 km/degree lat and ~108 km/degree long
# so this is about 11 km around guam 
# https://www.johndcook.com/how_big_is_a_degree.html
guam_coord = [13.4, 144.8]
cut = 0.16
print((111*0.16), (108*0.16))
guam_lat_cuts = [guam_coord[0]-cut, guam_coord[0]+cut]
guam_lon_cuts = [guam_coord[1]-cut, guam_coord[1]+cut]

# get the regions near guam to mask out and altitudes (not BL)
null_lat = np.where((awas_param_noblcorr.GGLAT > guam_lat_cuts[0]) 
                    & (awas_param_noblcorr.GGLAT < guam_lat_cuts[1]))
null_lon = np.where((awas_param_noblcorr.GGLON > guam_lon_cuts[0]) 
                    & (awas_param_noblcorr.GGLON < guam_lon_cuts[1]))
null_alt = np.where((awas_param_noblcorr.GGALT < 4000))

# get intersection, index list to mask 
near_guam = np.intersect1d(null_lat, null_lon, null_alt)

# get the opposite of that index, NOT near guam 
#not_guam = np.arange(0, len(toga_param.GGALT.values), dtype = float)
#not_guam[near_guam]=np.nan

not_guam = np.arange(0, len(awas_param_noblcorr.GGALT.values))
not_guam = np.delete(not_guam, near_guam)

17.76 17.28


In [10]:
# keep only non-guam values - AIRCRAFT PARAM 
awas_param = xr.Dataset()
for ii in awas_param_noblcorr.coords:
    param = awas_param_noblcorr[ii]
    param_noguam = param[not_guam].values 
    awas_param[ii] = param_noguam

# keep only non-guam values - TRACERS 
awas_trc = xr.Dataset()
for ii in awas_trc_noblcorr.data_vars:
    trc = awas_trc_noblcorr[ii]
    trc_noguam = trc[not_guam]
    awas_trc[ii] = trc_noguam


In [11]:
### ---------------------- ADD AC PARAMETERS AS COORDINATES ---------------------- ###
# ---- add parameters as coordinates 
for ii in awas_param.coords:
    my_coord = awas_param[ii]
    awas_trc.coords[ii] = my_coord

In [ ]:
### ---------------------- SAVE ---------------------- ###
### pickle it! 
awas_trc.to_netcdf(path = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_trace_gases_twp.nc')
print ('finished saving tracers')


In [27]:
###########################################################################################
###########################################################################################
###########################################################################################

In [23]:
###########################################################################################
### ---------------------- REPLACE TRACERS WITH ZEROS ---------------------- ###
# duplicate 
awas_trc_replace = awas_trc

myvars = ['C2H6S_Dimethyl_sulfide', 'CHBr2Cl_Dibromochloromethane', 
          'C4H9NO3_2_Butyl_nitrate', 'C4H9NO3_n_Butyl_nitrate']
mylows = [0.1, 0.07, 0.02, 0.03]

# dms 
dms = awas_trc_replace.C2H6S_Dimethyl_sulfide.values
dms_zloc = np.argwhere(dms == 0)
dms_replace = dms
dms_replace[dms_zloc] = (mylows[0]*0.8)

awas_trc_replace.C2H6S_Dimethyl_sulfide.values = dms_replace

#dibromomethane 
dibrom = awas_trc_replace.CHBr2Cl_Dibromochloromethane.values
dibrom_zloc = np.argwhere(dibrom == 0)
dibrom_replace = dibrom
dibrom_replace[dibrom_zloc] = (mylows[1]*0.8)

awas_trc_replace.CHBr2Cl_Dibromochloromethane.values = dibrom_replace

# 2 butyl 
but2 = awas_trc_replace.C4H9NO3_2_Butyl_nitrate.values
but2_zloc = np.argwhere(but2 == 0)
but2_replace = but2
but2_replace[but2_zloc] = (mylows[2]*0.8)

awas_trc_replace.C4H9NO3_2_Butyl_nitrate.values = but2_replace

# n butyl 
butn = awas_trc_replace.C4H9NO3_n_Butyl_nitrate.values
butn_zloc = np.argwhere(butn == 0)
butn_replace = butn
butn_replace[butn_zloc] = (mylows[3]*0.8)

awas_trc_replace.C4H9NO3_n_Butyl_nitrate.values = butn_replace


In [26]:
### ---------------------- SAVE ---------------------- ###
### pickle it! 
awas_trc_replace.to_netcdf(path = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_trace_gases_twp_replace.nc')
print ('finished saving tracers')

finished saving tracers


In [167]:
### ---------------------- GET INFO ---------------------- ###
#### 
# C2H6S_Dimethyl_sulfide
# C4H9NO3_2_Butyl_nitrate
# CHBr2Cl_Dibromochloromethane
# C4H9NO3_n_Butyl_nitrate
myvars = ['C2H6S_Dimethyl_sulfide', 'C4H9NO3_2_Butyl_nitrate', 
           'CHBr2Cl_Dibromochloromethane', 'C4H9NO3_n_Butyl_nitrate']

for vv in myvars:
    print(vv)
    v = awas_trc[vv].values
    print(np.nanmean(v))
    len_total = len(np.ndarray.flatten(v))
    len_zero = len(np.ndarray.flatten(np.argwhere(v == 0)))
    len_nan = len(np.where(~np.isfinite(v))[0])
    print(len_total, len_zero, len_nan)

    zero_loc = np.argwhere(v == 0)
    var_nozero = v
    var_nozero[zero_loc] = np.nan
    var_vals = var_nozero[~np.isnan(var_nozero)]
    var_min = np.min(var_vals)
    print(var_min)
    var_replace = v
    var_replace[zero_loc] = (var_min*0.8)
    print(np.nanmean(var_replace))

C2H6S_Dimethyl_sulfide
3.5897574
541 76 170
0.1
3.6061451
C4H9NO3_2_Butyl_nitrate
0.61172867
541 15 3
0.02
0.61217475
CHBr2Cl_Dibromochloromethane
0.17462824
541 1 3
0.07
0.17473234
C4H9NO3_n_Butyl_nitrate
0.95024115
541 129 2
0.03
0.95598525
